# Implementing a Convolutional Neural Network on the CPU from Scratch
I could implement a CNN using pytorch's layers with trivial ease `nn.Conv2d` is all that is required. However, I'd like to demonstrate a detailed understanding of the fundamental mechanics.

In [1]:
import torch 
from torch import optim, nn
import torch.nn.functional as F

from PIL import Image
import numpy as np

import pandas as pd
import os

In [2]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test_df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


It's clear each row represents an image. I want to convert the underlying data structure, `numpy.ndarray` to a PNG file so that I may augment and manipulate the images with imaging libraries, like `PIL`.

In [24]:
"""
for i in range(len(train_df)):
    arr = np.array(train_df.iloc[i,1:]).reshape(28,28)
    arr = arr.astype('uint8')

    img = Image.fromarray(arr, mode='L')

    label = train_df.iloc[i,0] 
    
    os.makedirs(f'./train/{label}/', exist_ok=True)
    img.save(f'./train/{label}/train_{i}.png')
"""

In [8]:
"""
for i in range(len(test_df)):
    arr = np.array(test_df.iloc[i,:]).reshape(28,28)
    arr = arr.astype('uint8')
    
    img = Image.fromarray(arr, mode='L')
    
    os.makedirs('./test/', exist_ok=True)
    img.save(f'./test/test_{i}.png')
"""

### Let's establish baseline accuracy
A simple linear model with zero data augmentation.

In [5]:
full_ds = []

for i in range(len(train_df)):
    arr = np.array(train_df.iloc[i,1:])
    arr = arr.astype(np.float32) / 255
    features_t = torch.from_numpy(arr)
    
    label_t = torch.tensor(train_df.iloc[i, 0])
    label_t = F.one_hot(label_t, num_classes=10)
    label_t = label_t.float()
    
    full_ds.append((features_t, label_t))

In [6]:
cutoff = int(len(full_ds) * 0.8)
train_ds = full_ds[:cutoff]
val_ds = full_ds[cutoff:]

In [7]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=False)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=64, shuffle=False)


In [8]:
input_neurons = len(test_df.columns)
classes = len(train_df["label"].unique())

In [9]:
dense_model = nn.Sequential(nn.Linear(input_neurons, input_neurons//2),
                           nn.ReLU(),
                           nn.Linear(input_neurons//2, input_neurons//4),
                           nn.ReLU(),
                           nn.Linear(input_neurons//4, input_neurons//8),
                           nn.ReLU(),
                           nn.Linear(input_neurons//8, classes))    # I want to softmax these outputs

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(dense_model.parameters(), lr=1e-3)

In [11]:
from learner import ClassificationLearner
clearn = ClassificationLearner(model=dense_model,
                              optimizer=optimizer,
                              criterion=criterion,
                              train_dl=train_dl,
                              val_dl=val_dl)

In [12]:
from rich.console import Console
console = Console()
try:
    clearn.fit(25, save_best=True)
except Exception as e:
    console.print_exception(show_locals=True)

Output()

### Our model did train pretty well :)

In [13]:
clearn.best_accuracy

0.941047619047619

### Let's submit to see test set accuracy

In [17]:
test_ds = []
for i in range(len(test_df)):
    arr = np.array(test_df.iloc[i,:])
    arr = arr.astype(np.float32) / 255
    features_t = torch.from_numpy(arr)
    
    test_ds.append(features_t)

In [25]:
with open("./submission.csv", 'w') as f:
    f.write("ImageId,Label")
    
for num, sample in enumerate(test_ds):
    one_hot_pred = clearn.get_preds(sample)
    one_hot_pred.unsqueeze_(-1)
    _, pred_class = torch.max(one_hot_pred)
    
    with open("./submission.csv", 'a') as f:
        f.write(f"{num},{pred_class}")

TypeError: iteration over a 0-d tensor